In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

%matplotlib inline 
sns.set_style("white")

In [ ]:
## select the dataset
#df=pd.read_csv('../Final/Berlin/merge_data_files/Entire_home_apt.csv' ,low_memory=False,index_col=0) 
df=pd.read_csv('../Final/Berlin/2020-06-13_listings.csv',low_memory=False)

In [ ]:
# Listings segmentation using K-Means Clustering

In [ ]:
pd.set_option("display.max_columns", 106) 
df.head()

In [ ]:
df.price

In [ ]:
df.price=pd.to_numeric(df.price.str.replace('$','').str.replace(',',''))
df.cleaning_fee=pd.to_numeric(df.cleaning_fee.str.replace('$','').str.replace(',',''))
df.cleaning_fee

In [ ]:
df.cleaning_fee.fillna(0, inplace = True) 

In [ ]:
df.cleaning_fee

In [ ]:
df

In [ ]:
#function for locking a dataset 
# column its a string
#argument is a list 


def lock_data(column, argument):
    d=pd.concat([df.loc[df[column]==i] for i in argument])
    return d

In [ ]:
df=lock_data('room_type', ['Entire home/apt'])

In [ ]:
df

In [ ]:
# find the the mean for each neighborhood

df_mean=df.groupby('neighbourhood_cleansed').agg({'price': 'median', 'id' : 'count'})
df_mean

In [ ]:
df=pd.merge(df, df_mean, on='neighbourhood_cleansed', how='left')

In [ ]:
df['host_is_superhost'].value_counts()

In [ ]:
df['host_is_superhost'].loc[df['host_is_superhost'] == 't'] = 1 
df['host_is_superhost'].loc[df['host_is_superhost'] != 1] = 0 

In [ ]:
df['host_is_superhost'].value_counts()

In [ ]:
# dummies variables 0 when host is abroad

df['foreign_host']=~df['host_location'].str.contains('Germany|DE|Deutschland', na=True ,regex=True)
df['foreign_host']=df['foreign_host'].astype(int)


In [ ]:

df['foreign_host']

In [ ]:
df['foreign_host'].dtype

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
X = df[["latitude", "longitude",'calculated_host_listings_count']]
        #,'price','number_of_reviews']]


In [ ]:
inertias = {}

for k in range(1,15):
    cl = KMeans(n_clusters=k).fit(X)
    inertias[k] = cl.inertia_
plt.figure(figsize=(15,6))
sns.lineplot(x=list(inertias.keys()),
             y=list(inertias.values()));
plt.title("Optimal number of clusters");

In [ ]:
## model

clusters = KMeans(n_clusters=3)
clusters.fit(X)
data_cluster=X.assign(cluster=clusters.predict(X))
data_cluster

In [ ]:
sns.scatterplot(data=X,
                x="longitude",
                y="latitude", 
                hue=clusters.predict(X));

In [ ]:
df=pd.concat([df, data_cluster['cluster'].astype(str)], axis=1)

In [ ]:
df

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color=clusters.predict(X).astype(str),
                        hover_name="host_name",
                        hover_data=["host_id","calculated_host_listings_count",'neighbourhood_cleansed',
                                    "number_of_reviews", 'price_x','price_y','foreign_host'],
                        #size='foreign_host',
                        color_discrete_sequence=["orange","blue",'black', 'pink'],
                        zoom=10, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("../Final/big_players2.html")

In [ ]:
df= df.merge(pd.get_dummies(df.cluster, prefix='group_investor', prefix_sep='_',drop_first=True),
              left_index=True,
              right_index=True)

In [ ]:
fig, ax = plt.subplots(nrows=1,
                       ncols=3,
                       figsize=(18,6));

sns.distplot(df["price_x"],
             ax=ax[0]);
ax[0].set_title("price");

sns.distplot(df["reviews_per_month"],
             ax=ax[1]);
ax[1].set_title("reviews_per_month");

sns.distplot(df["review_scores_location"],
             ax=ax[2]);
ax[2].set_title("review_scores_location");

In [ ]:
fig, ax = plt.subplots(nrows=1,
                       ncols=3,
                       figsize=(18,6));

sns.distplot(df["availability_30"],
             ax=ax[0]);
ax[0].set_title("availability_30");

sns.distplot(df["availability_60"],
             ax=ax[1]);
ax[1].set_title("availability_60");

sns.distplot(df["availability_90"],
             ax=ax[2]);
ax[2].set_title("availability_90");

In [ ]:
#Create a dataset for cluster and try to find the hotspots


super_cluster=df[['id_x','neighbourhood_cleansed','review_scores_location', 'foreign_host',
                'host_location','host_is_superhost','price_x','cleaning_fee',
                'minimum_maximum_nights','availability_30','availability_60','availability_90',
                'number_of_reviews_ltm','calculated_host_listings_count','group_investor_1','group_investor_2']]
super_cluster

In [ ]:
super_cluster=super_cluster.assign(total_price=super_cluster.price_x+super_cluster.cleaning_fee)

In [ ]:
super_cluster =super_cluster.groupby('neighbourhood_cleansed', as_index=False).agg({'total_price': 'median', 'id_x' : 'count', 
                                                                                     'review_scores_location':'mean','foreign_host': sum,
                                                     'availability_30':'mean','availability_60':'mean',
                                                     'availability_90':'mean', 'number_of_reviews_ltm': 'mean',
                                                     'calculated_host_listings_count':'mean',
                                                    'group_investor_1':sum,'group_investor_2':sum })

super_cluster

In [ ]:
the_X = super_cluster[['total_price','id_x','review_scores_location','foreign_host',
                        'number_of_reviews_ltm','calculated_host_listings_count','group_investor_1','group_investor_2']]


In [ ]:
inertias = {}

for k in range(1,25):
    cl = KMeans(n_clusters=k).fit(the_X)
    inertias[k] = cl.inertia_
plt.figure(figsize=(15,6))
sns.lineplot(x=list(inertias.keys()),
             y=list(inertias.values()));
plt.title("Optimal number of clusters");

In [ ]:

clusters = KMeans(n_clusters=3)
clusters.fit(the_X)
super_cluster=super_cluster.assign(cluster=clusters.predict(the_X))
super_cluster



In [ ]:
super_cluster.loc[super_cluster.cluster==4]

In [ ]:
super_cluster.loc[super_cluster.cluster==3]

In [ ]:
super_cluster.loc[super_cluster.cluster==2]

In [ ]:
super_cluster.loc[super_cluster.cluster==1]

In [ ]:
super_cluster.loc[super_cluster.cluster==0]

In [ ]:
super_cluster.rename(columns = {'neighbourhood_cleansed':'neighbourhood'}, inplace = True)

In [ ]:
super_cluster

In [ ]:
import json

with open('berlin-neighbourhoods.geojson',encoding="utf-8") as json_file:
    data = json.load(json_file)

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(super_cluster, geojson=data, locations='neighbourhood', 
                           featureidkey="properties.neighbourhood", color=super_cluster.cluster.astype(str),
                                        
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 52.5, "lon": 13.4},
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("../Final/cluster_airbnb_hotspot.html")

In [ ]:
(super_cluster[['total_price','id_x','review_scores_location','foreign_host','number_of_reviews_ltm',
                'calculated_host_listings_count',
               'group_investor_1','group_investor_2','cluster']]
                 .groupby("cluster").describe().transpose())

In [ ]:
len(data)

In [ ]:
data['features'][0]

In [4]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\johnz\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\johnz\\AppData\\Local\\Temp\\pip-install-yiwnm0p2\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\johnz\\AppData\\Local\\Temp\\pip-install-yiwnm0p2\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\johnz\AppData\Local\Temp\pip-pip-egg-info-2yg_cqmd'
         cwd: C:\Users\johnz\AppData\Local\Temp\pip-install-yiwnm0p2\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command out

  Using cached geopandas-0.8.1-py2.py3-none-any.whl (962 kB)
  Using cached Shapely-1.7.1-cp38-cp38-win_amd64.whl (1.0 MB)
  Using cached Fiona-1.8.17.tar.gz (1.3 MB)


In [1]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'